In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("salesdaily.csv")
df

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour,Weekday Name
0,01-02-2014,0.00,3.670,3.40,32.40,7.0,0.0,0.0,2.00,2014,1,248,Thursday
1,01-03-2014,8.00,4.000,4.40,50.60,16.0,0.0,20.0,4.00,2014,1,276,Friday
2,01-04-2014,2.00,1.000,6.50,61.85,10.0,0.0,9.0,1.00,2014,1,276,Saturday
3,01-05-2014,4.00,3.000,7.00,41.10,8.0,0.0,3.0,0.00,2014,1,276,Sunday
4,01-06-2014,5.00,1.000,4.50,21.70,16.0,2.0,6.0,2.00,2014,1,276,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,10-04-2019,7.34,5.683,2.25,22.45,13.0,0.0,1.0,1.00,2019,10,276,Friday
2102,10-05-2019,3.84,5.010,6.00,25.40,7.0,0.0,0.0,0.33,2019,10,276,Saturday
2103,10-06-2019,4.00,11.690,2.00,34.60,6.0,0.0,5.0,4.20,2019,10,276,Sunday
2104,10-07-2019,7.34,4.507,3.00,50.80,6.0,0.0,10.0,1.00,2019,10,276,Monday


In [3]:
df_new = df.melt(id_vars=['datum', 'Year', 'Month', 'Hour', 'Weekday Name'], var_name='Drug', value_name='Quantity')
df_new.head()

,datum,Year,Month,Hour,Weekday Name,Drug,Quantity
0,01-02-2014,2014,1,248,Thursday,M01AB,0.0
1,01-03-2014,2014,1,276,Friday,M01AB,8.0
2,01-04-2014,2014,1,276,Saturday,M01AB,2.0
3,01-05-2014,2014,1,276,Sunday,M01AB,4.0
4,01-06-2014,2014,1,276,Monday,M01AB,5.0


In [4]:
df_new.shape

(16848, 7)

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_new['Weekday Name'] = le.fit_transform(df_new['Weekday Name'])
df_new['Drug'] = le.fit_transform(df_new['Drug'])
df_new

,datum,Year,Month,Hour,Weekday Name,Drug,Quantity
0,01-02-2014,2014,1,248,4,0,0.00
1,01-03-2014,2014,1,276,0,0,8.00
2,01-04-2014,2014,1,276,2,0,2.00
3,01-05-2014,2014,1,276,3,0,4.00
4,01-06-2014,2014,1,276,1,0,5.00
...,...,...,...,...,...,...,...
16843,10-04-2019,2019,10,276,0,7,1.00
16844,10-05-2019,2019,10,276,2,7,0.33
16845,10-06-2019,2019,10,276,3,7,4.20
16846,10-07-2019,2019,10,276,1,7,1.00


In [6]:
Q1 = df_new['Quantity'].quantile(0.25)
Q3 = df_new['Quantity'].quantile(0.75)
Q1,Q3

(np.float64(1.81775), np.float64(8.0))

In [7]:
IQR = Q3 - Q1
IQR

np.float64(6.18225)

In [8]:
lower_lim = Q1 - 1.5*IQR
upper_lim = Q3 + 1.5*IQR
lower_lim,upper_lim

(np.float64(-7.4556249999999995), np.float64(17.273375))

In [9]:
df_new_no_out = df_new[df_new['Quantity'] < 17.27]
df_new_no_out

,datum,Year,Month,Hour,Weekday Name,Drug,Quantity
0,01-02-2014,2014,1,248,4,0,0.00
1,01-03-2014,2014,1,276,0,0,8.00
2,01-04-2014,2014,1,276,2,0,2.00
3,01-05-2014,2014,1,276,3,0,4.00
4,01-06-2014,2014,1,276,1,0,5.00
...,...,...,...,...,...,...,...
16843,10-04-2019,2019,10,276,0,7,1.00
16844,10-05-2019,2019,10,276,2,7,0.33
16845,10-06-2019,2019,10,276,3,7,4.20
16846,10-07-2019,2019,10,276,1,7,1.00


In [10]:
train = df_new_no_out[df_new_no_out['Year'] < 2019]
test = df_new_no_out[df_new_no_out['Year'] >= 2019]

print(train.shape)
print(test.shape)

(12871, 7)
(2004, 7)


In [11]:
X_train = train.drop(['Hour','Quantity','datum'],axis = 1)
y_train = train['Quantity']

X_test = test.drop(['Hour','Quantity','datum'],axis = 1)
y_test = test['Quantity']

In [12]:
import xgboost as xgb
reg = xgb.XGBRegressor(n_estimators = 1000,early_stopping_rounds = 50, learning_rate = 0.005)
reg.fit(X_train,y_train,
       eval_set = [(X_train,y_train),(X_test,y_test)],
       verbose = 10)

[0]	validation_0-rmse:3.77079	validation_1-rmse:3.95352
[10]	validation_0-rmse:3.68695	validation_1-rmse:3.86350
[20]	validation_0-rmse:3.60921	validation_1-rmse:3.78049
[30]	validation_0-rmse:3.53734	validation_1-rmse:3.70430
[40]	validation_0-rmse:3.47093	validation_1-rmse:3.63554
[50]	validation_0-rmse:3.40964	validation_1-rmse:3.57251
[60]	validation_0-rmse:3.35310	validation_1-rmse:3.51397
[70]	validation_0-rmse:3.30092	validation_1-rmse:3.46084
[80]	validation_0-rmse:3.25290	validation_1-rmse:3.41130
[90]	validation_0-rmse:3.20867	validation_1-rmse:3.36521
[100]	validation_0-rmse:3.16793	validation_1-rmse:3.32257
[110]	validation_0-rmse:3.13038	validation_1-rmse:3.28386
[120]	validation_0-rmse:3.09580	validation_1-rmse:3.24866
[130]	validation_0-rmse:3.06403	validation_1-rmse:3.21607
[140]	validation_0-rmse:3.03487	validation_1-rmse:3.18602
[150]	validation_0-rmse:3.00809	validation_1-rmse:3.15810
[160]	validation_0-rmse:2.98351	validation_1-rmse:3.13266
[170]	validation_0-rmse:2

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.005, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [13]:
from sklearn.metrics import mean_squared_error
y_pred =reg.predict(X_test)
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
rmse

np.float64(2.8343275208950365)

In [14]:
test['Quantity Predictions'] = y_pred
test

C:\Users\telan\AppData\Local\Temp\ipykernel_11696\3726821739.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Quantity Predictions'] = y_pred


,datum,Year,Month,Hour,Weekday Name,Drug,Quantity,Quantity Predictions
1825,01-01-2019,2019,1,276,5,0,0.00,4.645026
1826,01-02-2019,2019,1,276,6,0,5.33,4.930352
1827,01-03-2019,2019,1,276,4,0,4.33,4.383024
1828,01-04-2019,2019,1,276,0,0,7.00,4.540607
1829,01-05-2019,2019,1,276,2,0,8.01,5.436758
...,...,...,...,...,...,...,...,...
16843,10-04-2019,2019,10,276,0,7,1.00,2.406292
16844,10-05-2019,2019,10,276,2,7,0.33,2.400013
16845,10-06-2019,2019,10,276,3,7,4.20,2.763385
16846,10-07-2019,2019,10,276,1,7,1.00,2.400013


In [15]:
def predict_sales(start_date,end_date,drug):
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    df_test = pd.DataFrame(index=dates)
    df_test['Year'] = df_test.index.year
    df_test['Month'] = df_test.index.month
    df_test['Weekday Name'] = df_test.index.weekday
    df_test['Drug'] = drug
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df_test['Weekday Name'] = le.fit_transform(df_test['Weekday Name'])
    df_test['predicted_quantity'] = reg.predict(df_test)
    return df_test

In [16]:
predict_sales(start_date = '2024-12-01',end_date = '2024-12-31',drug = 4)

,Year,Month,Weekday Name,Drug,predicted_quantity
2024-12-01,2024,12,6,4,8.541022
2024-12-02,2024,12,0,4,8.628966
2024-12-03,2024,12,1,4,8.353431
2024-12-04,2024,12,2,4,7.555841
2024-12-05,2024,12,3,4,6.678446
2024-12-06,2024,12,4,4,7.739897
2024-12-07,2024,12,5,4,7.405951
2024-12-08,2024,12,6,4,8.541022
2024-12-09,2024,12,0,4,8.628966
2024-12-10,2024,12,1,4,8.353431


In [17]:
predict_sales(start_date = '2024-10-01',end_date = '2024-10-31',drug = 0)

,Year,Month,Weekday Name,Drug,predicted_quantity
2024-10-01,2024,10,1,0,4.790873
2024-10-02,2024,10,2,0,5.387918
2024-10-03,2024,10,3,0,5.577176
2024-10-04,2024,10,4,0,4.537754
2024-10-05,2024,10,5,0,4.607937
2024-10-06,2024,10,6,0,4.496473
2024-10-07,2024,10,0,0,4.531419
2024-10-08,2024,10,1,0,4.790873
2024-10-09,2024,10,2,0,5.387918
2024-10-10,2024,10,3,0,5.577176
